# Stock NeurIPS2018 Part 1. Data
This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*. 

This is the first part of the NeurIPS2018 series, introducing how to use FinRL to fetch and process data that we need for ML/RL trading.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages

In [ ]:
# ## install required packages
# !pip install swig
# !pip install wrds
# !pip install pyportfolioopt
# ## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

In [1]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools

# Part 2. Fetch data

[yfinance](https://github.com/ranaroussi/yfinance) is an open-source library that provides APIs fetching historical data form Yahoo Finance. In FinRL, we have a class called [YahooDownloader](https://github.com/AI4Finance-Foundation/FinRL/blob/master/finrl/meta/preprocessor/yahoodownloader.py) that use yfinance to fetch data from Yahoo Finance.

**OHLCV**: Data downloaded are in the form of OHLCV, corresponding to **open, high, low, close, volume,** respectively. OHLCV is important because they contain most of numerical information of a stock in time series. From OHLCV, traders can get further judgement and prediction like the momentum, people's interest, market trends, etc.

## Data for a single ticker

Here we provide two ways to fetch data with single ticker, let's take Apple Inc. (AAPL) as an example.

### Using yfinance

In [2]:
aapl_df_yf = yf.download(tickers = "aapl", start='2020-01-01', end='2020-01-31')

[*********************100%***********************]  1 of 1 completed


In [3]:
aapl_df_yf.head()

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2020-01-02 00:00:00+00:00,72.876122,75.087502,75.150002,73.797501,74.059998,135480400
2020-01-03 00:00:00+00:00,72.167603,74.357498,75.144997,74.125000,74.287498,146322800
2020-01-06 00:00:00+00:00,72.742661,74.949997,74.989998,73.187500,73.447502,118387200
2020-01-07 00:00:00+00:00,72.400551,74.597504,75.224998,74.370003,74.959999,108872000
2020-01-08 00:00:00+00:00,73.565208,75.797501,76.110001,74.290001,74.290001,132079200


### Using FinRL

In FinRL's YahooDownloader, we modified the data frame to the form that convenient for further data processing process. We use adjusted close price instead of close price, and add a column representing the day of a week (0-4 corresponding to Monday-Friday).

In [4]:
aapl_df_finrl = YahooDownloader(start_date = '2020-01-01',
                                end_date = '2020-01-31',
                                ticker_list = ['aapl']).fetch_data()

[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (20, 8)


In [5]:
aapl_df_finrl.head()

,date,open,high,low,close,volume,tic,day
0,2020-01-02,72.876122,75.087502,75.150002,74.059998,135480400,aapl,3
1,2020-01-03,72.167603,74.357498,75.144997,74.287498,146322800,aapl,4
2,2020-01-06,72.742661,74.949997,74.989998,73.447502,118387200,aapl,0
3,2020-01-07,72.400551,74.597504,75.224998,74.959999,108872000,aapl,1
4,2020-01-08,73.565208,75.797501,76.110001,74.290001,132079200,aapl,2


## Data for the chosen tickers

In [6]:
config_tickers.DOW_30_TICKER

['AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'KO',
 'JPM',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'CRM',
 'VZ',
 'V',
 'WBA',
 'WMT',
 'DIS',
 'DOW']

In [7]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2021-10-29'

In [8]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (94301, 8)


In [9]:
df_raw.head()

,date,open,high,low,close,volume,tic,day
0,2009-01-02,2.737005,3.241071,3.251429,3.067143,746015200,AAPL,4
1,2009-01-02,41.816147,58.990002,59.080002,58.590000,6547900,AMGN,4
2,2009-01-02,15.014067,19.330000,19.520000,18.570000,10955700,AXP,4
3,2009-01-02,33.941101,45.250000,45.560001,42.799999,7010200,BA,4
4,2009-01-02,30.602570,46.910000,46.980000,44.910000,7117200,CAT,4


# Part 3: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

Hear let's take **MACD** as an example. Moving average convergence/divergence (MACD) is one of the most commonly used indicator showing bull and bear market. Its calculation is based on EMA (Exponential Moving Average indicator, measuring trend direction over a period of time.)

In [14]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

Successfully added technical indicators


[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (3228, 8)
Successfully added vix
Successfully added turbulence index


In [15]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [17]:
processed_full[:100]

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,2.737005,3.241071,3.251429,3.067143,746015200.0,4.0,0.000000,3.565522,2.829121,100.000000,66.666667,100.000000,3.067143,3.067143,39.580002,0.0
1,2009-01-02,AMGN,41.816147,58.990002,59.080002,58.590000,6547900.0,4.0,0.000000,3.565522,2.829121,100.000000,66.666667,100.000000,58.590000,58.590000,39.580002,0.0
2,2009-01-02,AXP,15.014067,19.330000,19.520000,18.570000,10955700.0,4.0,0.000000,3.565522,2.829121,100.000000,66.666667,100.000000,18.570000,18.570000,39.580002,0.0
3,2009-01-02,BA,33.941101,45.250000,45.560001,42.799999,7010200.0,4.0,0.000000,3.565522,2.829121,100.000000,66.666667,100.000000,42.799999,42.799999,39.580002,0.0
4,2009-01-02,CAT,30.602570,46.910000,46.980000,44.910000,7117200.0,4.0,0.000000,3.565522,2.829121,100.000000,66.666667,100.000000,44.910000,44.910000,39.580002,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,2009-01-07,DIS,19.804075,23.180000,23.889999,23.889999,12300700.0,2.0,0.039208,24.522499,22.462501,100.000000,-38.681878,12.094781,23.492500,23.492500,40.290001,0.0
154,2009-01-07,GS,64.605095,84.500000,87.849998,86.910004,12465500.0,2.0,0.129619,91.762560,81.632441,64.763956,-60.580318,42.208903,86.697500,86.697500,40.290001,0.0
155,2009-01-07,HD,16.634567,24.570000,25.160000,24.870001,19857300.0,2.0,0.067429,25.977447,22.472554,96.210202,39.193458,51.186783,24.225000,24.225000,40.290001,0.0
156,2009-01-07,HON,22.958591,32.843578,33.959023,33.959023,5969983.0,2.0,0.087388,35.003647,30.702581,100.000000,36.240193,22.217434,32.853114,32.853114,40.290001,0.0


# Part 4: Save the Data

### Split the data for training and trading

In [18]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

83897
9715


### Save data to csv file

For Colab users, you can open the virtual directory in colab and manually download the files.

For users running on your local environment, the csv files should be at the same directory of this notebook.

In [20]:
train.to_csv('train_data.csv')
trade.to_csv('trade_data.csv')